In [7]:
using JLD2, NeuralVerification, ReachabilityAnalysis, Plots, TaylorModels
using ReachabilityAnalysis: solve, Interval

In [2]:
@load "/home/sguadalupe/Documents/Julia/AI/b7_2.jld2" controller;

In [3]:
@taylorize function f!(dx, x, p, t)
    x₁, x₂, x₃, w, u = x

    dx[1] = x₃^3 - x₂ + w
    dx[2] = x₃
    dx[3] = u
    dx[4] = zero(w) # w
    dx[5] = zero(u) # u
    return dx
end


solver = MaxSens(resolution=100.0, tight=true)

X₀ = Hyperrectangle(low=[0.35, 0.45, 0.25], high=[0.45, 0.55, 0.35])
W₀ = Interval(0, 0) # Interval(-0.01, 0.01)

U₀ = Interval(0, 0)
U₀ = forward_network(solver, controller, X₀)

Q₀ = X₀ × W₀ × U₀
#Q₀ = overapproximate(Q₀, Hyperrectangle)
#Q₀ = split(Q₀, 4*ones(Int, 5))
prob = @ivp(x' = f!(x), dim: 5, x(0) ∈ Q₀);

Tsample = 0.5 # sect
t0 = 0.0
ti = t0
k = 1

sol1 = solve(prob, tspan=(ti, ti+k*Tsample), alg=TMJets(abs_tol=1e-16, orderT=8, orderQ=1));
sol1z = overapproximate(sol1, Zonotope);

UndefVarError: UndefVarError: Interval not defined

In [42]:
function nsolve(func, X₀, vars, nn::Network, solver, tspan, sampling_time)
    U₀ = forward_network(solver, nn, X₀)
    Q₀ = X₀
    for var in vars
        Q₀ = Q₀ × var
    end
    t = 0.0
    sol = Vector(undef, Int(tspan/sampling_time))
    for i = 1:Int(tspan/sampling_time)
        P₀ = Q₀ × U₀
        prob = @ivp(func.s, x(0) ∈ P₀);
        sol[i] = solve(prob, tspan=(t, t+sampling_time), alg=TMJets(abs_tol=1e-16, orderT=8, orderQ=1));
        t += sampling_time
        Q₀ = set(Projection(overapproximate(sol[i][end], Hyperrectangle).X, vars=[1:dim(Q₀)]))
        U₀ = forward_network(solver, nn, Q₀)
    end
    return sol
end

nsolve (generic function with 1 method)

In [43]:
prob = @ivp(x' = f!(x), dim: 5, x(0) ∈ X₀);

In [44]:
X₀ = Hyperrectangle(low=[0.35, 0.45, 0.25], high=[0.45, 0.55, 0.35])
W₀ = Interval(-0.01, 0.01)
sol = nsolve(prob, X₀, [W₀], controller, MaxSens(), 2, 0.5);

MethodError: MethodError: no method matching Projection(::Hyperrectangle{Float64,StaticArrays.SArray{Tuple{5},Float64,1,5},StaticArrays.SArray{Tuple{5},Float64,1,5}}; vars=UnitRange{Int64}[1:4])
Closest candidates are:
  Projection(!Matched::ReachabilityAnalysis.AbstractLazyReachSet; vars) at /home/sguadalupe/.julia/dev/ReachabilityAnalysis/src/Flowpipes/reachsets.jl:516
  Projection(!Matched::ReachabilityAnalysis.AbstractLazyReachSet, !Matched::Tuple{Vararg{M,D}}) where {D, M<:Integer} at /home/sguadalupe/.julia/dev/ReachabilityAnalysis/src/Flowpipes/reachsets.jl:491 got unsupported keyword argument "vars"
  Projection(!Matched::ReachabilityAnalysis.AbstractLazyReachSet, !Matched::Tuple{Vararg{M,D}}, !Matched::Bool) where {D, M<:Integer} at /home/sguadalupe/.julia/dev/ReachabilityAnalysis/src/Flowpipes/reachsets.jl:491 got unsupported keyword argument "vars"
  ...

In [55]:
overapproximate(sol[1][end], Hyperrectangle).X

Hyperrectangle{Float64,StaticArrays.SArray{Tuple{5},Float64,1,5},StaticArrays.SArray{Tuple{5},Float64,1,5}}([-5694.904363138294, -5.07631646165418, -22.605265846616717, 0.0, -91.62106338646686], [6341.020477666118, 5.892122961726759, 23.318491846907037, 0.01, 0.7264520005806361])

In [34]:
?ReachSet

search: ReachSet ShiftedReachSet TemplateReachSet SparseReachSet



```
ReachSet{N, ST<:LazySet{N}} <: AbstractLazyReachSet{N}
```

Type that wraps a reach-set using a `LazySet` as underlying representation.

### Fields

  * `X`  – set
  * `Δt` – time interval

### Notes

A `ReachSet` is a struct representing (an approximation of) the reachable states for a given time interval. The type of the representation is `ST`, which may be any subtype LazySet. For efficiency reasons, `ST` should be concretely typed.

By assumption the coordinates in this reach-set are associated to the integers `1, …, n`. The function `vars` returns such tuple.
